# Session #9 Homework

In [1]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.lite as tflite
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Question 1

Convert this model from Keras to TF-Lite format.  
What's the size of the converted model?

In [2]:
model = keras.models.load_model("dogs_cats_10_0.687.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open("dogs_cats_10_0.687.tflite", "wb") as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\mdrak\AppData\Local\Temp\tmptb7ynh0p\assets


In [3]:
ls

 Volume in drive D is Files
 Volume Serial Number is 5850-A0F0

 Directory of D:\Development\mlcamp\09-serverless

29 Nov 2021  23:35    <DIR>          .
28 Nov 2021  15:42    <DIR>          ..
28 Nov 2021  18:58    <DIR>          .ipynb_checkpoints
29 Nov 2021  19:39               287 Dockerfile
28 Nov 2021  15:43        89.758.304 dogs_cats_10_0.687.h5
29 Nov 2021  23:35        44.865.956 dogs_cats_10_0.687.tflite
29 Nov 2021  23:35             8.095 homework.ipynb
28 Nov 2021  15:43             4.559 homework.md
29 Nov 2021  19:48             1.337 lambda_function.py
               6 File(s)    134.638.538 bytes
               3 Dir(s)  30.332.022.784 bytes free


The size of the converted model is approximately **43MB**.

## Question 2

To be able to use this model, we need to know the index of the input and the index of the output.  
What's the output index for this model?

In [4]:
interpreter = tflite.Interpreter(model_path="dogs_cats_10_0.687.tflite")
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

print(f"Input index: {input_index}, Output index: {output_index}")

Input index: 0, Output index: 13


Output index is **13**.

## Preparing the Image

In [5]:
from io import BytesIO
from urllib import request
from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != "RGB":
        img = img.convert("RGB")
    img = img.resize(target_size, Image.NEAREST)
    return img

The target size of the image should be **(150,150)**.

In [6]:
url = "https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg"
downloaded_image = download_image(url)
prepared_image = prepare_image(downloaded_image, (150,150))

## Question 3

Now we need to turn the image into an numpy array and pre-process it.  
After the pre-processing, what's the value in the first pixel, the R channel?

In [7]:
x = np.array(prepared_image, dtype="float32")
x = np.array([x])

datagen = ImageDataGenerator(rescale=1./255)
X = datagen.flow(x, batch_size=1)
X = X.next()

In [8]:
X[0][0][0]

array([0.7058824, 0.7725491, 0.7490196], dtype=float32)

The value of the first pixel is **0.7058824**.

## Question 4

Now let's apply this model to this image. What's the output of the model?

In [9]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
preds[0].tolist()[0]

0.769909143447876

Prediction of the model is **0.7699**.